In [ ]:
import os
import numpy as np
import ast
import pandas as pd
from scipy.ndimage import uniform_filter1d 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from collections import Counter
from tqdm import tqdm
sns.set_theme(style="white")
plt.figure(figsize=(10, 8))
colors = ['#66c2a5', '#fc8d62']


plt.rcParams.update({
    'font.size': 30,
    'axes.labelsize': 30,
    'xtick.labelsize': 30,
    'ytick.labelsize': 30,
    'legend.fontsize': 30,
    'axes.titlesize': 30
})

task = "aime24"
models = ["qwen3-32b", "qwen3-14b","qwen3-8b","qwen3-4b","qwen3-1-7b", "qwen3-0-6b"]

In [2]:
sparse_method = "blocktopk"
if sparse_method == "topk":
    fname_sign = """_local0_topk{budget}"""
elif sparse_method == "blocktopk":
    fname_sign = """_local0_block{block_size}_blocktopk{block_topk}"""
else:
    raise ValueError(f"Invalid sparse method: {sparse_method}")

In [ ]:
sparse_points = []
dense_points = []
genlens = [2048, 4096, 6144, 8192, 10240, 12288, 14336, 16384, 18432, 20480, 22528, 24576, 26624, 28672, 30720, 32768]

root_dir = "/home/rsadhukh/ranajoy/sparse_scaling/Kinetics/cost_model"
for model in models:
    if os.path.exists(f"{root_dir}/{task}/dense/{task}_{model}_dense_genlen_tradeoff.csv"):
        dense_df = pd.read_csv(f"{root_dir}/{task}/dense/{task}_{model}_dense_genlen_tradeoff.csv")
    else:
        dense_df = None
    
    sparse_df = []
    for budget in [128, 256, 512, 1024, 2048]:
        block_size = 16384 // budget
        block_topk = budget // block_size
        
        if sparse_method == "topk":
            fname = f"{root_dir}/{task}/{sparse_method}/{task}_{model}{fname_sign.format(budget=budget)}_genlen_tradeoff.csv"
        elif sparse_method == "blocktopk":
            fname = f"{root_dir}/{task}/{sparse_method}/{task}_{model}{fname_sign.format(block_size=block_size, block_topk=block_topk)}_genlen_tradeoff.csv"
        else:
            raise ValueError(f"Invalid sparse method: {sparse_method}")
        
        if os.path.exists(fname):
            sparse_df.append(pd.read_csv(fname))

    if len(sparse_df) > 0:
        sparse_df = pd.concat(sparse_df)
    else:
        sparse_df = None
    
    min_cost = 4500e-2
    max_cost = 1e5
    cost_points = np.logspace(np.log10(min_cost), np.log10(max_cost), 100)
    
    covs = []
    if dense_df is not None:
        for i, cost in enumerate(cost_points):
            best_covs = []
            for query_id in dense_df["query_id"].unique():
                query_df = dense_df[dense_df["query_id"] == query_id]
                best_coverage = query_df[query_df["total_cost"] <= cost]["coverage"]
                if len(best_coverage) > 0:
                    best_cov = best_coverage.max()
                    best_covs.append(best_cov)
                else:
                    best_covs.append(0)
            covs.append(np.mean(best_covs))
        
        dense_points.append((model, "full", covs, cost_points))
        
    sparse_covs = []
    if sparse_df is not None:
        for i, cost in enumerate(cost_points):
            best_covs = []
            best_ntrials = []
            best_budgets = []
            median_generation_lengths = []
            for query_id in sparse_df["query_id"].unique():
                query_df = sparse_df[sparse_df["query_id"] == query_id]
                best_coverage = query_df[query_df["total_cost"] <= cost]["coverage"]
                if len(best_coverage) > 0:
                    cur_best_cov = best_coverage.max()
                    best_covs.append(cur_best_cov)
                else:
                    best_covs.append(0)
            sparse_covs.append(np.mean(best_covs))
        sparse_points.append((model, "sparse", sparse_covs, cost_points))

In [5]:
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib import cm

def get_shades(base_color, n_shades=6, darken_factor=0.4):
    """Return n_shades darker shades of the base_color."""
    base_rgb = np.array(mcolors.to_rgb(base_color))
    dark_rgb = base_rgb * darken_factor  # darker version
    shades = [mcolors.to_hex((1 - alpha) * base_rgb + alpha * dark_rgb)
              for alpha in np.linspace(0, 1, n_shades)]
    return shades

shades_dense = get_shades(colors[1], n_shades=len(models))
shades_sparse = get_shades(colors[0], n_shades=len(models))
shades_sparse = shades_sparse[::-1]
shades_dense = shades_dense[::-1]

In [ ]:
plt.clf() 

window_size = 3 

plt.figure(figsize=(12, 8))
# sns.set_theme(style="whitegrid")
all_dense_points = []
# for i, ((model, label, covs, cost_points), color) in enumerate(zip(dense_points, shades_dense)) 
print(len(dense_points)) 
for i, ((model, label, covs, cost_points), color) in enumerate(zip(dense_points, shades_dense)): 
    c = np.array(cost_points) / 4500 
    smooth_covs = uniform_filter1d(covs, window_size) 
    plt.plot(c, smooth_covs, linestyle=":", linewidth=2,alpha=0.5, color=colors[i]) 
    all_dense_points += [(model, p[0], p[1]) for p in zip(smooth_covs, cost_points)]

all_sparse_points = []
# for (model, budget, covs, cost_points), color in zip(sparse_points, shades_sparse): 
print(len(sparse_points)) 
for i, ((model, budget, covs, cost_points), color) in enumerate(zip(sparse_points, shades_sparse)): 
    c = np.array(cost_points) / 4500 
    smooth_covs = uniform_filter1d(covs, window_size) 
    plt.plot(c, smooth_covs, linestyle="--", linewidth=2,alpha=0.5, color = colors2[i]) 
    all_sparse_points += [(model, p[0], p[1]) for p in zip(smooth_covs, cost_points)] 


model_sizes = {
    "qwen3-32b": 32.76,
    "qwen3-8b": 8.19,
    "qwen3-4b": 4.02,
    "qwen3-14b": 14.33,
    "qwen3-1-7b": 2.03,
    "qwen3-0-6b": 0.752
} 

model_sizesinverse = {
    v: k for k, v in model_sizes.items() 
} 

models2 = ["qwen3-32b", "qwen3-14b","qwen3-8b","qwen3-4b","qwen3-1-7b", "qwen3-0-6b"] 
models2 = models2[: : -1] 


all_sparse_points = sorted(all_sparse_points, key=lambda x: x[0])
all_dense_points = sorted(all_dense_points, key=lambda x: x[0])
all_costs = [p[2] for p in all_sparse_points]
best_model_sizes = [] 
best_model_sizesdense = [] 
best_sparse_acc = []
best_dense_acc = []
max_cost = 4e4
min_cost= 20
cost_points = np.logspace(np.log10(min_cost), np.log10(max_cost), 100)
x_data = []
y_data = []
for c in cost_points:
    valid_points = [p for p in all_sparse_points if  p[2] <= c] 
    if not valid_points:
        best_model_sizes.append(0.752) 
        continue
    best_point = max(valid_points, key=lambda x: x[1])
    best_sparse_acc.append(best_point[1])
    best_model_sizes.append(model_sizes[best_point[0]])
    x_data.append(c) 
print(best_model_sizes) 

for c in cost_points:
    valid_points = [p for p in all_dense_points if  p[2] <= c]
    if not valid_points: 
        best_model_sizesdense.append(0.752) 
        continue
    best_point = max(valid_points, key=lambda x: x[1])
    best_dense_acc.append(best_point[1]) 
    best_model_sizesdense.append(model_sizes[best_point[0]]) 
    y_data.append(c) 
print(best_model_sizesdense) 

x_data = np.array(x_data) / 4500
y_data = np.array(y_data) / 4500 
print(len(x_data), len(y_data), len(best_sparse_acc), len(best_dense_acc)) 
colorsspecial = [] 
# colors2 = [] 

last_best_model = 0.752 
last_startidx = 0 
for i in range(len(x_data)): 
    if best_model_sizes[i] != last_best_model: 
        plt.plot(x_data[last_startidx : i + 1], best_sparse_acc[last_startidx : i + 1], color = colors2[models2.index(model_sizesinverse[last_best_model])], linewidth=4, zorder = 1) 
        plt.scatter(x_data[i], best_sparse_acc[i], color = colors2[models2.index(model_sizesinverse[last_best_model])], s=100, zorder = 2) 
        last_best_model = best_model_sizes[i] 
        last_startidx = i 
    else: 
        continue 
plt.plot(x_data[last_startidx : ], best_sparse_acc[last_startidx : ], color = colors2[models2.index(model_sizesinverse[last_best_model])], linewidth=4) 
# plt.scatter(x_data, best_sparse_acc, label=f"Pareto Frontier (Topk)", c = colorsspecial) 
last_best_model = 2.03 
last_startidx = 0 
for i in range(len(x_data)): 
    if best_model_sizesdense[i] != last_best_model: 
        plt.plot(y_data[last_startidx : i + 1], best_dense_acc[last_startidx : i + 1], color = colors[models2.index(model_sizesinverse[last_best_model])], linewidth=4, zorder = 1) 
        plt.scatter(y_data[i], best_dense_acc[i], color = colors[models2.index(model_sizesinverse[last_best_model])], s = 150, marker = '^', zorder = 2) 
        last_best_model = best_model_sizesdense[i] 
        last_startidx = i 
    else: 
        continue 
plt.plot(y_data[last_startidx : ], best_dense_acc[last_startidx : ], color = colors[models2.index(model_sizesinverse[last_best_model])], linewidth=4) 
# plt.plot(y_data, best_dense_acc, label=f"Pareto Frontier (Dense)", linewidth=4, color=colors[1]) 

t_thresh = 0.5
acc_thresh = 0.7

x_data_acc_diff = [i for i, p in enumerate(x_data) if p <= 0.5][-1]
sparse_acc_08 = best_sparse_acc[x_data_acc_diff]
dense_acc_08 = best_dense_acc[x_data_acc_diff]

y_data_time_diff = [i for i, p in enumerate(best_sparse_acc) if p <=0.6][-1]
sparse_time_08 = y_data[y_data_time_diff]
x_data_time_diff = [i for i, p in enumerate(best_dense_acc) if p <=0.6][-1]
dense_time_08 = x_data[x_data_time_diff] 

plt.xscale("log")

plt.xlabel("B200 seconds", fontsize=28)
plt.ylabel("Accuracy", fontsize=28)

plt.xticks(fontsize=28)
plt.yticks(fontsize=28)
# plt.xlim(0.01, 11)
plt.xlim(x_data[0], x_data[-1])
plt.subplots_adjust(left=0.1, right=0.98, top=0.98, bottom=0.1)
plt.tight_layout()
ax = plt.gca() 

plt.savefig(f"{task}_acc_vs_genlen_{sparse_method}.pdf")